In [1]:
from intake import open_catalog

cat = open_catalog("https://raw.githubusercontent.com/pangeo-data/pangeo-datastore/master/intake-catalogs/atmosphere.yaml")
ds  = cat["era5_hourly_reanalysis_single_levels_sa"].to_dask()

In [3]:
import xarray as xr
from intake import open_catalog
import numpy as np

# 1️⃣ 连接 ERA5 数据目录
cat = open_catalog("https://raw.githubusercontent.com/pangeo-data/pangeo-datastore/master/intake-catalogs/atmosphere.yaml")

# 打开单层面风速数据集
ds = cat["era5_hourly_reanalysis_single_levels_sa"].to_dask()

# 2️⃣ 查看数据结构
print("✅ 数据结构预览:")
print(ds)

print("\n📦 变量列表:")
print(list(ds.data_vars))

print("\n📏 坐标维度:")
print(ds.dims)



✅ 数据结构预览:
<xarray.Dataset> Size: 50TB
Dimensions:    (time: 350640, latitude: 721, longitude: 1440)
Coordinates:
  * time       (time) datetime64[ns] 3MB 1979-01-01 ... 2018-12-31T23:00:00
  * latitude   (latitude) float32 3kB 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * longitude  (longitude) float32 6kB 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
Data variables: (12/17)
    asn        (time, latitude, longitude) float64 3TB dask.array<chunksize=(31, 721, 1440), meta=np.ndarray>
    d2m        (time, latitude, longitude) float64 3TB dask.array<chunksize=(31, 721, 1440), meta=np.ndarray>
    e          (time, latitude, longitude) float64 3TB dask.array<chunksize=(31, 721, 1440), meta=np.ndarray>
    mn2t       (time, latitude, longitude) float64 3TB dask.array<chunksize=(31, 721, 1440), meta=np.ndarray>
    mx2t       (time, latitude, longitude) float64 3TB dask.array<chunksize=(31, 721, 1440), meta=np.ndarray>
    ptype      (time, latitude, longitude) float64 3TB dask.array<chunksize=(31, 

In [5]:
import xarray as xr
import numpy as np
from intake import open_catalog

# -------------------------------
# 1️⃣ 打开 ERA5 数据目录
# -------------------------------
cat = open_catalog("https://raw.githubusercontent.com/pangeo-data/pangeo-datastore/master/intake-catalogs/atmosphere.yaml")
ds = cat["era5_hourly_reanalysis_single_levels_sa"].to_dask()

print("✅ 成功读取 ERA5 数据集！")
print("包含变量：", list(ds.data_vars))

# -------------------------------
# 2️⃣ 提取 10 米风速分量
# -------------------------------
u10 = ds["u10"]
v10 = ds["v10"]

# -------------------------------
# 3️⃣ 计算风速大小
# -------------------------------
wind_speed = np.sqrt(u10**2 + v10**2)

# -------------------------------
# 4️⃣ 选取时间范围（1980–2020）
# -------------------------------
ds_sel = wind_speed.sel(time=slice("1980-01-01", "2020-12-31"))

# -------------------------------
# 5️⃣ 提取纬度范围（热带 -23.5° 到 23.5°）
# -------------------------------
lat_range = slice(-23.5, 23.5)
ds_tropics = ds_sel.sel(latitude=lat_range)

# -------------------------------
# 6️⃣ 处理经度范围
# ERA5 经度是 0–360，需要转换为对应太平洋范围 100–280E
# -------------------------------
lon_range = slice(100, 280)
ds_pacific = ds_tropics.sel(longitude=lon_range)

# -------------------------------
# 7️⃣ 输出为新的 Dataset
# -------------------------------
ds_out = xr.Dataset({
    "wind_speed": ds_pacific
})

ds_out.attrs["description"] = "10m wind speed over Tropical Pacific (100E–280E, 23.5S–23.5N), hourly ERA5 data (1980–2020)"
ds_out.attrs["source"] = "ERA5 reanalysis via Pangeo datastore"

# -------------------------------
# 8️⃣ 保存为 netCDF 文件
# -------------------------------
output_path = "ERA5_TropicalPacific_WindSpeed_1980_2020.nc"
ds_out.to_netcdf(output_path)
print(f"✅ 已保存提取后的热带太平洋风速数据: {output_path}")

# -------------------------------
# 9️⃣ 验证输出
# -------------------------------
test = xr.open_dataset(output_path)
print("✅ 验证通过，输出维度：", test.dims)


✅ 成功读取 ERA5 数据集！
包含变量： ['asn', 'd2m', 'e', 'mn2t', 'mx2t', 'ptype', 'ro', 'sd', 'sro', 'ssr', 't2m', 'tcc', 'tcrw', 'tp', 'tsn', 'u10', 'v10']


KeyboardInterrupt: 